# Winter 2025 134A Week 4

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

# Import all the functions from the Utility_functions.py file
from Utility_functions import *

## Bond Duration
In bond analysis, it is common to use yield instead of the interest rate. Duration measures the bond's sensitivity to changes in yield, which relates directly to price. Note the duration is often less than the duration of the maturity. The longer the duration is, the higher risk it will have against the change of the interest rate. There are two commonly used types of duration: **Macaulay duration** and **modified duration**. 

Macaulay duration is the weighted average time to receive the bond's cash flows, considering the yield. Suppose a financial instrument makes payments $m$ times per year, with the payment in period being $c_k$, and there are $n$ periods remaining. The Macaulay duration is defined as:
$$D = \frac{\sum_{k=1}^n \frac{k}{m}\frac{c_k}{(1+\lambda/m)^k}}{P}$$

Denote $P_k$ to be the present value of payment $c_k$ received in the period $k$. Then we have $P_k = \frac{c_k}{(1+\lambda/m)^k}$. Hence, we may write $D$ as 
$$D = \frac{\sum_{k=1}^n \frac{k}{m}P_k}{P}$$

Note that we also have $\frac{dP_k}{d\lambda} = -\frac{k}{m(1+\lambda/m)}P_k$. Let $P$ be the present value of the payment cashflow, which is also the price of the bond. Then we have $P = P_1+\cdots+P_k$, and thus 
$$\frac{dP}{d\lambda} = \sum_1^n \frac{dP_k}{d\lambda} = \sum_1^n\frac{-\frac{k}{m} P_k}{1+\lambda/m} = \frac{-D\cdot P}{1+\lambda/m} = -D_MP$$
We call $D_M = \frac{D}{1+\lambda/m}$ as the **modified duration**. Roughly speaking
$$\frac{\Delta P}{P} \approx -D_M\Delta \lambda$$

In the case where all coupon payments are identical (the normal case for bonds), there is an explicit formula for the Macaulay duration. For a bond with a coupon rate $c$ per period, yield $y$ per period, $m$ periods per year, and exactly $n$ periods remaining, the Macaulay duration is:
$$D = \frac{1+y}{my}-\frac{1+y+n(c-y)}{mc[(1+y)^n - 1] + my}$$

**Example**: 
Consider a bond with YTM 10%, and modified duration being 10 years. If the YTM increase to 11%, what can we say about the price of this bond

a) Increase by 1% \
b) Decrease by 1% 

**Solution:** The price of this bond will decrease by 10% and increase by 10%.

In [6]:
def macaulay_duration(c, y, m, n):
    """
    c is the coupon rate per period
    y is the yield per period
    m is the number of periods per year
    n is the number of periods remaining
    """
    D = (1+y)/(m*y) - (1 + y + n*(c-y))/(m*c * ((1+y)**n - 1) + m*y)
    return D

**Exercise 3.14**: 
(Duration) Find the price and duration of a 10-year, 8% bond that is trading at a yield of 10%. Assume that the coupon is paid semi-annually.

In [7]:
print(f"The duration is {macaulay_duration(c = 0.08/2, y = 0.1/2, m = 2, n = 20):.2f} years")

The duration is 6.84 years


## Duration of a Portfolio
Assume that a portfolio consists of two bonds with payment cash flow $(0, c_1, c_2,\ldots, c_n)$ and $(0, d_1,\ldots, d_n)$. We may extend one of the cash flow with zeros so that they have the same length. We want to compute the duration of this portfolio. Let $P^A_k$ be the present value of the payment $c_k$, and $P^B_k$ be the resent value of the payment $d_k$. Let $P^A$ be the price/present value of the first bond, and $P^B$ be the price of the second bond. Let $D^A$ and $D^B$ to be their durations. Then we have 
$$D^A = \frac{\sum_1^n t_kP^A_k}{P^A}$$
$$D^B = \frac{\sum_1^n t_kP^B_k}{P^B}$$

Let $D$ be the duration of this portfolio, and $P = P_A+P_B$ to be the present value of this portfolio. Then we have 
$$D = \frac{\sum_1^n t_k (P^A_k+P^B_k)}{P} = \frac{D^AP^A+D^BP^B}{P} = \frac{P^A}{P}D^A + \frac{P^B}{P}D^B$$

In other words, the duration of a portfolio is a weighted sum of the duration of each individual bonds, and the weight is the proportion of this bonds price over the price of this portfolio.

More generally, if a portfolio consists of $n$ fixed income investments with price $P_1,\ldots, P_n$, and each of them has a duration $D_1,\ldots, D_n$. Let $P = P_1+\cdots+P_n$ be the price of this portfolio, and $D$ be its duration. Then we have 
$$D = \frac{P_1}{P}D_1+\cdots+\frac{P_n}{P}D$$

## Immunization
The goal of **Immunization** is to protect a portfolio against the change in the interest rate. If we have an obligation of paying $F$ amount of money in $D$ years. Suppose that the prevailing interest is $r$. Then the YTM of a bond will be close to $r$. A zero coupon bond with face value $F$ and mature in $D$ years will serve the purpose. However, it is very common that there is no such a bond in the market. We may construct a portfolio of other bonds so that the present value of this portfolio will equal to the present value of the future payment. However, if the interest rate change, then our portfolio may not worth the same as the payment that we need to deliver. 

Then what we do is to construct a portfolio consisting of fixed income securities so that its present value matches with the present value of the future payment, its duration matches with $D$, which is the time we need to deliver the payment, or equivalently, the duration of the hypothetical zero-coupon bond which mature in $D$ years. In this case, if the interest rate change, then the present value of the future payment and the present value of our portfolio will have the same amount of change. 

In terms of math, we need to solve a portfolio $P$ satisfies:
$$\left\{
    \begin{aligned}
    &D_P = D\\
    &PV(P) = F
    \end{aligned}
    \right.$$

### Example:
(Bond selection) 
Consider the four bonds having annual payments as shown in the following table. They are traded to produce a 15\% yield. 

a) Determine the price of each bond. \
b) Determine the duration of each bond (not the modified duration). \
c) Which bond is most sensitive to a change in yield? \
d) Suppose you owe 2,000 at the end of 2 years. Concern about interest rate risk suggests that a portfolio consisting of the bonds and the obligation should be immunized. If
$V_A, V_B, V_C$, and $V_D$ are the total values of bonds purchased of types A,B,C, and D, respectively, what are the necessary constraints to implement the immunization? \
e) In order to immunize the portfolio, you decide to use bond C and one other bond. Which other bond should you choose? Find the amounts (in total value) of each of these to purchase. \
f) You decided in e) to use bond C in the immunization. Would other choices, including perhaps a combination of bonds, lead to lower total cost?

<center>

|        | Bond A | Bond B | Bond C | Bond D |
|--------|--------|--------|--------|--------|
| Year 1 | 100    | 50     | 0      | 1000   |
| Year 2 | 100    | 50     | 0      | 0      |
| Year 3 | 1100   | 1050   | 1000   | 0      |

</center>

#### Solution:

In [8]:
PV_a = PV(r = 0.15, cf = [0, 100, 100, 1100])
PV_b = PV(r = 0.15, cf = [0, 50, 50, 1050])
PV_c = PV(r = 0.15, cf = [0, 0, 0, 1000])
PV_d = PV(r = 0.15, cf = [0, 1000, 0, 0])
print(f"a) The price of each bonds are {PV_a: .2f}, {PV_b: .2f}, {PV_c:.2f}, {PV_d:.2f}")
d_a  = macaulay_duration(0.1, 0.15, 1, 3)
d_b  = macaulay_duration(0.05, 0.15, 1, 3)
d_c  = macaulay_duration(0, 0.15, 1, 3)
d_d  = macaulay_duration(0, 0.15, 1, 1)
print(f"b) The duration of each bonds are {d_a: .2f}, {d_b: .2f}, {d_c:.2f}, {d_d:.2f}")
print("c) The bond c is most sensitive to the change in yield")

a) The price of each bonds are  885.84,  771.68, 657.52, 869.57
b) The duration of each bonds are  2.72,  2.84, 3.00, 1.00
c) The bond c is most sensitive to the change in yield


d) Suppose that we bought $V_A,\ldots, V_D$ units of those $4$ bonds. Then the present value and duration of this portfolio is 
$$P = V_A \cdot 0.885 + V_B \cdot 0.771 + V_C \cdot 0.657 + V_D \cdot 0.869$$
$$D = \frac{V_A \cdot 0.885}{P} \cdot 2.72+\cdots + \frac{V_D \cdot 0.869}{P}\cdot 1$$
Then a necessary constrain to implement the immunization is that the following system of linear equations has at least a solution
$$\begin{cases}
\frac{2000}{1.15^2} = V_A \cdot 0.885 + V_B \cdot 0.771 + V_C \cdot 0.657 + V_D \cdot 0.869 \\
2 = \frac{V_A \cdot 0.885}{2000/1.15^2} \cdot 2.72+\cdots + \frac{V_D \cdot 0.869}{2000/1.15^2}\cdot 1 
\end{cases} $$

e) We may use bond C and bond D for computation. Then we have to solve the equation 
$$\begin{cases}
\frac{2000}{1.15^2} = V_C \cdot 0.657 + V_D \cdot 0.869 \\
2 = \frac{V_C \cdot 0.657}{2000/1.15^2} \cdot 3 + \frac{V_D \cdot 0.869}{2000/1.15^2}\cdot 1 
\end{cases} $$

By solving this equation, we get $V_C = 1150.9, V_D = 870.131$. In terms of their price, we need to spent $V_C \cdot 0.657 = 756.15$ to purchase bond C and $V_D\cdot 0.869 = 756.15$ to buy bond $D$. 

f) No, all choices will have the same price, which is the present value of the 2000 payment in 2 years. 

## Term Structure
The YTM of a bond with different mature dates are usually not the same. The bonds with longer mature dates usually have a higher YTM. We denote the **spot rate** $s_t$ to be the annual interest rate of money held from now, until the time $t$. The spot rate $s_t$ is usually computed from the YTM of a zero-coupon bond which mature in time t. In the term structure, we are assuming that the money held for different time will have different interest rates. Thus, if we assume yearly compounding, then when computing the present value of a cashflow, we use $d_1 = \frac{1}{1+s_1}$ to discount the payment received in year $1$, and $d_2 = \frac{1}{(1+s_2)^2}$ to discount the payment received in year 2. If we assume the continuous compounding, then $d_1 = e^{-s_1}$, $d_2 = e^{-2s_2}$. 

## Forward Rates
Since the interest rate for holding the money for 1 year is different from holding the money for 2 years. We also expect that interest rate for borrowing the money for 1 year now is different from borrowing the money for 1 year next year. We call the latter as the **forward rates**. More precisely, the forward rate $f_{i, j}$ with $j>i$ are the interest rates for money to be borrowed from time $i$ to time $j$ in the future, but under terms agreed upon today. Under the yearly compounding assumption, the forward rates can be computed from the equation that the present values are the same:
$$(1+s_j)^j = (1+s_i)^i(1+f_{i,j})^{j-i}$$
By solving this equation, we get 
$$f_{i,j} = \left(\frac{(1+s_j)^j}{(1+s_i)^i}\right)^{1/(j-i)}-1$$

If we assume continuous compounding, then we have 
$$e^{js_j} = e^{is_i}e^{(j-i)f_{i,j}}$$
which implies that 
$$js_j = is_i + (j-i)f_{i,j}$$
By solving this equation, we get 
$$f_{i,j} = \frac{js_j-is_i}{j-i}$$


## Short Rates
Given the spot rates $(s_1, s_2,\ldots)$. We define the **short rates** at time $k$ as $r_k = f_{k, k+1}$, and $r_0 = s_1$. The short rates $r_k$ is the interest rate for the 1-year term loan at time $k$.

Then we have the equation 
$$(1+s_k)^k = (1+r_0)\cdots(1+r_k)$$

$$(1+f_{i,j})^{j-1} = (1+r_i)\cdots(1+r_j)$$

Given the short rates, we can also determine the spot rates uniquely.


## Exercise 4.1
(One forward rate)
If the spot rates for 1 and 2 years are $s_1$ = 6.3% and $s_2$ = 6.9%, what is the forward rate $f_{1,2}$? Assume continuous compounding for the computation. 

### Solution:
It can be easily computed using the formula:
$$(1+s_2)^2 = (1+s_1)\times (1+f_{1,2})$$

In [9]:
f_12 = (2 * 0.069 - 0.063)/(2-1)
print(f"The forward rate f_12 is {f_12*100:.2f}%")

The forward rate f_12 is 7.50%


## Exercise 4.5
(Instantaneous rates)
Let s(t), $0 < t < \infty$, denote a spot rate curve; that is, the present value of a dollar to be received at time t is $e^{-s(t)t}$, For  $t_1<t_2$, let $f(t_1, t_2)$ be the forward rate between $t_1$ and $t_2$ implied by the given spot rate curve.

- (a) Find an expression for $f(t_1, t_2)$.\
- (b) Let $r(t) = \lim\limits_{t_2\rightarrow t}f(t, t_2)$. We can call $r(t)$ the **instantaneous interest rate** at time $t$. Show that $r(t) = s(t) +s'(t)t$. \
- (c) Suppose an amount $x_0$ is invested in a bank account at t = 0 which pays the instantaneous rate of interest r(t) at all t (compounded). Then the bank balance $x(t)$ will satisfy $dx(t)/dt = r(t)x(t)$. Find an expression for x(t).

### Solution:
- a) From the definition of forward rates, we have 
$$e^{s(t_1)t_1} = e^{s(t_2)t_2}e^{f(t_1, t_2)(t_2-t_1)}$$
By solving $f(t_1, t_2)$, we get
$$f(t_1, t_2) = \frac{t_2s(t_2)-t_1s(t_1)}{t_2-t_1}$$ 

- b) Using the L'Hospital Rule, we have
$$r(t) = \lim\limits_{t_2\rightarrow t} f(t, t_2) =  \lim\limits_{t_2\rightarrow t}\frac{t_2s(t_2)-ts(t)}{t_2-t} = \lim\limits_{t_2\rightarrow t} \frac{s(t_2)+t_2s'(t_2) - 0}{1} = s(t)+ts'(t)$$


- c) We have 
$$\frac{dx(t)}{x(t)} = r(t)dt = (s(t)+ts'(t))dt = d(ts(t))$$
Integrating both sides gives 
$$\ln(x(t)) = ts(t)+C$$
and we have 
$$x(t) = e^Ce^{ts(t)}$$
By substituting $t=0$, we get $e^C = x_0$, and thus, we have 
$$x(t) = x_0e^{ts(t)}$$

## Exercise 4.7
(Bond taxes) An investor is considering the purchase of 10-year U.S. Treasury bonds and plans to hold them to maturity. Federal taxes on coupons must be paid during the year they are received, and tax must also be paid on the capital gain realized at maturity (defined as the difference between face value and original price). Federal bonds are exempt from state taxes. This investor’s federal tax bracket rate is t = 30%, as it is for most individuals. There are two bonds that meet the investor’s requirements. Bond 1 is a 10-year, 10% bond with a price (in decimal form) of $P_1$ = 92.21. Bond 2 is a 10-year, 7% bond with a price of $P_2$ = 75.84. Based on the price information contained in those two bonds, the investor would like to compute the theoretical price of a hypothetical 10-year zero coupon bond that had no coupon payments and required tax payment only at maturity equal in amount to 30% of the realized capital gain (the face value minus the original price). This theoretical price should be such that the price of this bond and those of bonds 1 and 2 are mutually consistent on an after-tax basis. Find this theoretical price, and show that it does not depend on the tax rate t. (Assume all cash flows occur at the end of each year.)

### Solution:
Let $P_1, P_2$ be the price of the 10% bond and 7% bond respectively. Suppose that $P_0$ is the price of the zero coupon 10 year bond, and $t$ is the tax rate. We suppose that the zero coupon bond is constructed by using $x_1$ unit of the $10%$ bond, and $x_2$ unit of the $7%$ bond, and the face values are \$100. In order to cancel the coupon payment, we must have:
- (1): No annual coupon paid: 
$$x_1(1-t)\cdot 10 + x_2(1-t)\cdot 7 = 0$$
which is equivalent to 
$$10x_1+7x_2 = 0$$
- (2): The after tax payment at expiry to match, hence, we have 
$$x_1(100-(100-P_1)t) + x_2(100 - (100-P_2)t) = 100-(100-P_0)t$$
Note the price of this zero coupon bond satisfies $P_0 = x_1P_1+x_2P_2$. By substituting this to the above equality, we get 
$$x_1+x_2 = 1$$
Hence, the $x_1, x_2$ satisfies the equation 
$$\begin{cases}10x_1+7x_2 = 0 \\ x_1+x_2 = 1\end{cases}$$
We solve that $x_1 = -7/3, x_2 = 10/3$, and the price of the zero coupon bond is $x_1P_1+x_2P_2 = 37.64$, which does not depends on the tax rate $t$. 

## Exercise 4.9
(Flat Forwards) Show explicitly that if the spot rate curve is flat (with $s(k) = r$ for all $k$), then all the forward rates also equal $r$.

### Solution:

This is just plug in the formula of forward rates with $s_i = s_j = r$. Thus,
$$ f_{i,j} = \frac{(j-i)r}{j-i} = r$$

## Running Present Value
We consider a cashflow $(x_0, x_1,\ldots, x_n)$, where $x_i$ is received in year $i$. Suppose that the spot rates are $(s_1,s_2,\ldots, s_n)$, and we define the discount factor $d_k = d_{0, k} = \frac{1}{(1+s_k)^k}$, which is the discount factor from time k to time 0. We may also define the discount factor $d_{j,k} = \frac{1}{(1+f_{j, k})^{k-j}}$, which is the discount factor from time $k$ to time $j$. Let $PV(0)$ be the present value of this cashflow at time $0$, then we have 
$$PV(0) = x_0+x_1d_1+\cdots+x_kd_k$$

Let $PV(k)$ to be the present value of the remaining cashflow at time k. In other words, $PV(k)$ is the present value of the cashflow $(x_k, x_{k+1},\ldots, x_n)$ under the spot rates at time $k$, which is $(f_{k, k+1}, f_{k, k+2},\ldots, f_{k, k+n})$. Then we have 
$$PV(k) = x_k+x_{k+1}d_{k, k+1}+\cdots+x_{n}d_{k, n}$$

Then, we have the following formula for computing $PV(0)$ recursively
$$\begin{cases}
PV(n) = x_n \\
PV(k) = x_k+d_{k, k+1}PV(k+1)
\end{cases}$$

## Exercise 4.11
(Running PV example)
A (yearly) cash flow stream is $x = (—40, 10,10, 10, 10, 10, 10)$. The spot rates are (5.0, 5.3, 5.6, 5.8, 6.0, 6.1) 
- (a) Find the current discount factors $d_{0, k}$, and use them to determine the (net) present value of the stream. 
- (b) Find the series of expectations dynamics short-rate discount factors, and use the running present value method to evaluate the stream.

### Solution:
We use different discounting factors corresponding to different spot rates.

In [10]:
# Write the code to calculate the discount factors
cashflow = [-40, 10, 10, 10, 10, 10, 10]
spot_rates = [5, 5.3, 5.6, 5.8, 6, 6.1]
discount_factors_0 = []
for k in range(len(spot_rates)):
    discount_factors_0.append(1/ (1 + spot_rates[k]/100)**(k+1))
discount_factors_0_formatted = [ '%.3f' % elem for elem in discount_factors_0]
print("a) The current discount rates d_{0,k} are ", discount_factors_0_formatted)


a) The current discount rates d_{0,k} are  ['0.952', '0.902', '0.849', '0.798', '0.747', '0.701']


In [11]:

table_npv_methods = pd.DataFrame(index=['', '', '', ' ', ' ', ' ', ' '])

table_npv_methods['Year'] = [0, 1, 2, 3, 4, 5, 6]
table_npv_methods['Cashflow'] = cashflow 
table_npv_methods['Spot'] = [''] + spot_rates
table_npv_methods['Discount Factor'] = [1] + discount_factors_0
table_npv_methods['Present Value'] = table_npv_methods['Cashflow'] * table_npv_methods['Discount Factor']
npv = sum(table_npv_methods['Present Value'])
table_npv_methods.loc['Sum'] = ['', ' ',  ' ', ' ', npv] 

display(table_npv_methods)
print(f"The net present value of this cashflow is {npv:.3f}\n")


,Year,Cashflow,Spot,Discount Factor,Present Value
,0,-40,,1.0,-40.000000
,1,10,5,0.952381,9.523810
,2,10,5.3,0.901869,9.018686
,3,10,5.6,0.849197,8.491966
,4,10,5.8,0.7981,7.981000
,5,10,6,0.747258,7.472582
,6,10,6.1,0.700983,7.009833
Sum,,,,,9.497877


The net present value of this cashflow is 9.498



- (b): We compute the present value by the running PV methods, then we have the following:

In [12]:
# Calculate the present value of the cashflow
print(f"PV({6}) = {cashflow[-1]:.2f}")
pv_last_step = cashflow[-1]
for k in reversed(range(0, len(cashflow) - 1)):
    discount_factor = (1 + spot_rates[k-1]/100)**k / (1 + spot_rates[k]/100) ** (k+1)
    new_pv = cashflow[k] + pv_last_step * discount_factor
    print(f"PV({k}) = {cashflow[k]:.2f} + {pv_last_step:.3f} * {discount_factor:.4f} = {new_pv:.3f}")
    pv_last_step = new_pv

PV(6) = 10.00
PV(5) = 10.00 + 10.000 * 0.9381 = 19.381
PV(4) = 10.00 + 19.381 * 0.9363 = 28.146
PV(3) = 10.00 + 28.146 * 0.9398 = 36.453
PV(2) = 10.00 + 36.453 * 0.9416 = 44.324
PV(1) = 10.00 + 44.324 * 0.9470 = 51.973
PV(0) = -40.00 + 51.973 * 0.9524 = 9.498


## Fisher-Weil Duration
We define the duration for a cashflow in a similar way under the term structure. Under the continuous compounding model, we consider a spot rate curve given by $s_t$, where the discount factor payment received in year $t$ is $e^{-ts_t}$. The **Fisher-Weil Duration** of a cashflow $(x_{t_0},\ldots, x_{t_n})$ is defined by 
$$D_{FW} = \frac{1}{PV}\sum_0^n t_kx_{t_k}e^{-s_{t_k}t_k}$$ 
This is exactly the weighted sum of the time we receive each payment, and the weights are the ratio if the present value of each payment over the total present value of the cashflow. 

We call $s_t+\lambda$ as a **parallel shift** of the spot rate curve. In other words, we are adding $\lambda$ to all spot rates. We may represent the present value as a function of $\lambda$ as 
$$P(\lambda) = \sum_0^n x_{t_k}e^{-t_k(s_{t_k}+\lambda)}$$
Then, we have 
$$\frac{dP(\lambda)}{d\lambda}\Big|_{\lambda = 0} = -D_{FW}/P(0)$$

In other words, the Fisher-Weil duration measures the sensitivity of the price relative to parallel shift in the spot rates.

### Quasi-modified duration
Suppose that the interest compounds $m$ times every year. Then we may define the **Quasi-modified duration** in a similar way, as the sensitivity to the parallel shift of the spot rates. Let $(x_0,\ldots, x_n)$ be a cashflow, where the payment $x_k$ is received in the period $k$. Let $(s_1,\ldots, s_n)$ be the spot rates of each period, quoted as annual interest. Then we have
$$D_M = -\frac{1}{P(0)}\frac{dP(\lambda)}{d\lambda}\Big|_{\lambda = 0} = \frac{1}{PV}\sum_1^n \left(\frac{k}{m}\right)x_k\left(1+\frac{s_k}{m}\right)^{-(k+1)}$$

Note that this is also a weighted sum of the time that we receive each payment. The weights are almost the ratio of the present value of each payment over the present value of the total cashflow, except that we multiplied $(1+\frac{s_k}{m})^{-(k+1)}$ instead of $(1+\frac{s_k}{m})^{-k}$ for discounting. 

### Example
Suppose that the spot rates are given by $(7.67, 8.27, 8.81, 9.31, 9.75, 10.16, 10.52, 10.85, 11.15, 11.42, 11.67, 11.89)$. We consider two bonds, where bond A is a 12-year 6\% bond, and bond B is a 5-year 10% bond. Assume that the coupon are paid annually. Compute price and the quasi-modified duration of the above two bonds. 

In [13]:
spot_rates = [7.67, 8.27, 8.81, 9.31, 9.75, 10.16, 10.52, 10.85, 11.15, 11.42, 11.67, 11.89]
cashflow_A = [0] + [6] * 11 + [106]
cashflow_B = [0] + [10] * 4 + [110]
#P_A = 65.95
#P_B = 101.66

discount_factors = [1]
for k in range(len(spot_rates)):
    discount_factors.append(1/ (1 + spot_rates[k]/100)**(k+1))

modified_discount_factors = [1]
for k in range(len(spot_rates)):
    modified_discount_factors.append(1/ (1 + spot_rates[k]/100)**(k+2))

year = list(range(12+1))
table = pd.DataFrame(index = ['']*13)
table['year'] = year
table['Spot rates'] = ['']+spot_rates
table['Bond A'] = cashflow_A
table['Bond B'] = cashflow_B + [0] * 7
table['Discount Factor'] = discount_factors
table['PV A'] = table['Bond A'] * table['Discount Factor']
table['PV B'] = table['Bond B'] * table['Discount Factor']
table['Modified Discount Factor'] = modified_discount_factors
table['PV\' A'] = table['Bond A'] * table['Modified Discount Factor'] * table['year']
table['PV\' B'] = table['Bond B'] * table['Modified Discount Factor'] * table['year']

PV_A = sum(table['PV A'])
PV_B = sum(table['PV B'])
P_derivative_A = sum(table['PV\' A'])
P_derivative_B = sum(table['PV\' B'])

table.loc['Sum'] = ['', '','', '', '', PV_A, PV_B, '', P_derivative_A , P_derivative_B] 
table.loc['Duration'] = ['', '','', '', '', '', '', '', P_derivative_A/PV_A, P_derivative_B/PV_B] 

display(table)

,year,Spot rates,Bond A,Bond B,Discount Factor,PV A,PV B,Modified Discount Factor,PV' A,PV' B
,0,,0,0,1.0,0.0,0.0,1.0,0.000000,0.000000
,1,7.67,6,10,0.928764,5.572583,9.287638,0.862602,5.175613,8.626022
,2,8.27,6,10,0.853068,5.118409,8.530681,0.787908,9.454898,15.758163
,3,8.81,6,10,0.776236,4.657414,7.762356,0.713386,12.840953,21.401589
,4,9.31,6,10,0.700423,4.202538,7.00423,0.640768,15.378422,25.630703
,5,9.75,6,110,0.628026,3.768154,69.082817,0.572233,17.166987,314.728097
,6,10.16,6,0,0.559573,3.357436,0.0,0.507964,18.286686,0.000000
,7,10.52,6,0,0.496494,2.978963,0.0,0.449234,18.867844,0.000000
,8,10.85,6,0,0.438646,2.631877,0.0,0.395712,18.994154,0.000000
,9,11.15,6,0,0.386202,2.317213,0.0,0.34746,18.762862,0.000000


## Exercise 4.15
(Stream immunization) 
A company faces a stream of obligations over the next 8 years as shown: where the numbers denote thousands of dollars. The spot rate curve is that of the following example. Try to find a portfolio, consisting of the two bonds described in the following example, that has the same present value as the obligation stream and is immunized against an additive shift in the spot rate curve.

| year | 1   | 2   | 3   | 4   | 5   | 6   | 7   | 8  |
|------|-----|-----|-----|-----|-----|-----|-----|----|
|      | 500 | 900 | 600 | 500 | 100 | 100 | 100 | 50 |

In the example, suppose that the spot rates are given by $(7.67, 8.27, 8.81, 9.31, 9.75, 10.16, 10.52, 10.85, 11.15, 11.42, 11.67, 11.89)$. We consider two bonds, where bond A is a 12-year 6\% bond, and bond B is a 5-year 10% bond. Assume that the coupon are paid annually. 

### Solution:
We first compute the price $P$, and the duration $D$ of this obligation. Then we solve the system of linear equations 
$$\begin{cases}
x_1P_A+x_2P_B = P \\ 
x_1\frac{P_A}{P}D_A+x_2\frac{P_B}{P}D_B = D
\end{cases}$$ 
where $P_A, P_B$ are the price of bond A, bond B computed from above, and $D_A, D_B$ are the quasi-modified duration of bond $A$ and bond $B$. The solution $x_1$ and $x_2$ means a portfolio with $x_1$ units of bond A, and $x_2$ units of bond B. 

In [14]:
cashflow = [0, 500, 900, 600, 500, 100, 100, 100, 50]
pv = 0
pv_derivative = 0
for i in range(len(cashflow)):
    pv += cashflow[i] * discount_factors[i]
    pv_derivative += i * cashflow[i] * modified_discount_factors[i]
    
print(f"The present value of the obligation is {pv:.2f}")
print(f'The duration of this obligation is {pv_derivative/pv:.2f} years')

The present value of the obligation is 2238.44
The duration of this obligation is 2.45 years


Then We solve the following system of linear equations
$$\begin{cases}
65.95x_1 + 101.67x_2 = 2238.44 \\
\frac{65.95\cdot 7.07}{2238.44}x_1 + \frac{101.67\cdot 3.80}{2238.44}x_2 = 2.45
\end{cases}$$

We see that $x_1 = -13.84$, $x_2 = 31.00$